In [ ]:
# Import relevant packages
from dataset import FabricDataset
from matplotlib import pyplot as plt
import albumentations as A
import os
from tqdm import tqdm
import numpy as np
import cv2
import random

# define root of single fabric contained in olp-dataset
root = "../olp-dataset/Textile_1"

# FabricDataset

Transforms passed to `transform` are applied only once during loading (e.g. for resizing images), whereas `uncached_transforms` are applied to every sample during each call to `FabricDataset.__getitem__`.
`load_mode` specifies whether front-light, back-light images, or both images should be loaded.
The dataset furthermore ensures that the same operations/augmentations are applied to front-light and back-light images.

In [ ]:
uncached_transforms = A.Compose([A.NoOp()])
load_transforms = A.Compose([A.Resize(512, 512)])
dset = FabricDataset(
    root=root,
    annFile=os.path.join(root, "dataset.json"),
    cache=False,
    uncached_transform=uncached_transforms,
    transform=load_transforms,
    load_mode="both")

A call to `FabricDataset.__getitem__` returns a `dict` containing the following key/value pairs:

                "image":            loaded and transformed image
                "mask":             semantic segmentation mask of the image
                "masks":            binary segmentation masks of all individual defect-instances
                "bboxes":           bounding boxes in coco-format of the individual defect-instances
                "instance_labels":  class labels of the individual defect-instances
                "textile":          the fabric the image belongs to
                "target":           the, possibly transformed, image-level class-label/targ
                
Let's load and visualize some example images

In [ ]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White
def visualize_bbox(img, bbox, color=BOX_COLOR, thickness=2, **kwargs):
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    return img

In [ ]:
for idx, sample in enumerate(tqdm(dset)):
    if idx == 100:
        break
    if (sample["target"][-1] != 0) or (random.random() < 0.05):
        fig, ax = plt.subplots(1,3)
        fl_image = np.copy(sample["image"][:,:,:-1])
        bl_image = np.copy(sample["image"][:,:,-1]) # backlight luminance is stored in the last channel of the image
        for bbox in sample["bboxes"]:
            visualize_bbox(fl_image, bbox)
            visualize_bbox(bl_image, bbox)
        ax[0].imshow(fl_image)
        ax[1].imshow(bl_image, cmap="gray")
        ax[2].imshow(sample["mask"])